In [14]:
import os
import sys
import torch
import mlflow
from pathlib import Path

# Add repository root to Python path so the model can find its source code
repo_root = Path(os.getcwd()).resolve().parent  # notebook is in notebooks/, go up one
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
    print(f"Added {repo_root} to Python path")

# You can get the run ID from the MLflow UI
experiment_id = "551694781745399147"  # The ID from the mlruns folder structure
run_id = "f29cd116d26c48d087831c7dc126ef97"

# Point MLflow at the repository root's mlruns directory
mlruns_dir = str(repo_root / "mlruns")
mlflow.set_tracking_uri(f"file://{mlruns_dir}")

print("MLflow tracking URI:", mlflow.get_tracking_uri())
print("mlruns root:", mlruns_dir)

# First, try to load directly from the model in mlruns/[experiment_id]/models/[model_id]
model_id = "m-02e631ce987947dcbe8221456ba5dda1"
model_path = os.path.join(mlruns_dir, experiment_id, "models", model_id)

if os.path.exists(os.path.join(model_path, "MLmodel")):
    print(f"Loading model from: {model_path}")
    loaded_model = mlflow.pytorch.load_model(model_path)
else:
    # Fall back to runs:/ URI which requires the run to be properly registered
    print(f"Model not found at {model_path}, trying runs:/ URI...")
    logged_model = f"runs:/{run_id}/model"
    try:
        loaded_model = mlflow.pytorch.load_model(logged_model)
    except Exception as e:
        print(f"runs:/ load failed: {e}")
        # Search for any MLmodel file referencing this run
        found = False
        for root, dirs, files in os.walk(mlruns_dir):
            if "MLmodel" in files:
                mlmodel_path = os.path.join(root, "MLmodel")
                try:
                    with open(mlmodel_path, "r") as f:
                        content = f.read()
                except Exception:
                    continue
                if f"run_id: {run_id}" in content:
                    model_dir = root
                    print(f"Found model MLmodel at {model_dir}; loading from there")
                    loaded_model = mlflow.pytorch.load_model(model_dir)
                    found = True
                    break
        if not found:
            raise Exception(f"Could not find model files for run {run_id} in {mlruns_dir}")

# Print the model's state dictionary which contains the parameters
print("\nModel loaded successfully. State dict:")
print(loaded_model.state_dict())

# Iterate the state dict to reliably access weights and biases
print("\nModel parameters:")
for name, tensor in loaded_model.state_dict().items():
    # Move tensor to CPU and convert to numpy for readable printing
    t = tensor.cpu().numpy() if isinstance(tensor, torch.Tensor) else tensor
    if "weight" in name:
        print(f"Weights (m) - {name}: {t}")
    if "bias" in name:
        print(f"Bias (c) - {name}: {t}")

Added /home/gilfoyle/data/play/ml/ml-pipeline-demo to Python path
MLflow tracking URI: file:///home/gilfoyle/data/play/ml/ml-pipeline-demo/mlruns
mlruns root: /home/gilfoyle/data/play/ml/ml-pipeline-demo/mlruns
Model not found at /home/gilfoyle/data/play/ml/ml-pipeline-demo/mlruns/551694781745399147/models/m-02e631ce987947dcbe8221456ba5dda1, trying runs:/ URI...


2025/11/07 04:11:17 WARNING mlflow.pytorch: Stored model version '2.4.0+cu121' does not match installed PyTorch version '2.8.0+cu128'
2025/11/07 04:11:17 WARNING mlflow.pytorch: Stored model version '2.4.0+cu121' does not match installed PyTorch version '2.8.0+cu128'



Model loaded successfully. State dict:
OrderedDict([('linear.weight', tensor([[3.1416, 2.8579, 2.9064, 2.7218, 3.2498]])), ('linear.bias', tensor([2.6683]))])

Model parameters:
Weights (m) - linear.weight: [[3.1415677 2.8579462 2.9063544 2.7218373 3.2498457]]
Bias (c) - linear.bias: [2.6683426]
